# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Get list of files in the event data folder using glob
file_path_list = glob.glob(os.path.join(filepath,'*'))

print(file_path_list)   

/Users/admin/Documents/projects/Sami222/Data Engineering Nanodegree Program/Data Modeling with Postgres/project2
['/Users/admin/Documents/projects/Sami222/Data Engineering Nanodegree Program/Data Modeling with Postgres/project2/event_data/2018-11-01-events.csv', '/Users/admin/Documents/projects/Sami222/Data Engineering Nanodegree Program/Data Modeling with Postgres/project2/event_data/2018-11-02-events.csv', '/Users/admin/Documents/projects/Sami222/Data Engineering Nanodegree Program/Data Modeling with Postgres/project2/event_data/2018-11-03-events.csv', '/Users/admin/Documents/projects/Sami222/Data Engineering Nanodegree Program/Data Modeling with Postgres/project2/event_data/2018-11-04-events.csv', '/Users/admin/Documents/projects/Sami222/Data Engineering Nanodegree Program/Data Modeling with Postgres/project2/event_data/2018-11-05-events.csv', '/Users/admin/Documents/projects/Sami222/Data Engineering Nanodegree Program/Data Modeling with Postgres/project2/event_data/2018-11-06-event

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Query 1 Description

I named the table created for this query `song_history` to reflect the data modelled in this table that is the history of heard songs per session.

In this query, I used `sessionId` as the partition key and `itemInSession` as clustering key.  Each partition is uniquely identified by `sessionId` while `itemInSession` was used to uniquely identify the rows within a partition to sort the data by the value of 4.

In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Create table for Query 1
# The data modelled in this table is the history of heard songs per session.
#
# This table contains data columns required for query 1, including:
#   artist, song, length, sessionId, and itemInSession. 
#
# The primary key is composed of sessionId as partition key, and itemInSession as clustering column. 
# Although multiple songs may be heard in one session, the combination of sessionId and itemInSession would 
# uniquely identify each heard song because only one song is heard at a time.
query = """CREATE TABLE IF NOT EXISTS song_history (
    sessionId INT, 
    itemInSession INT,
    artist TEXT,
    song TEXT,
    length FLOAT,
    PRIMARY KEY (sessionId, itemInSession)
)
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# Read the collected event data file to insert data into table song_history for answering query 1
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Query to insert data into song_history table
        query = "INSERT INTO song_history (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        # Choose sessionId (col#8), itemInSession (col#3), artist (col#0), song (col#9), and length (col#5).
        # Also convert string into int or float to match with column's data type.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# Now run Query 1 to verify the data was entered into the table
query = "SELECT artist, song, length FROM song_history WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Print the query result
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### Query 2 Description

I named the table created for this query `user_history` to reflect the data modelled in this table that is the history of heard songs per user.

In this query, I used `sessionId` and `userId` as the composite partition key and `itemInSession` as clustering key.  Each partition is uniquely identified by the combination of `sessionId` and `userId` while `itemInSession` was used to uniquely identify the rows within a partition to sort user's listened songs in ascending order.

In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# Create table for Query 2
# The data modelled in this table is the history of heard songs per user.
#
# This table contains data columns required for query 2, including:
#   artist, song, itemInSession, firstName, lastName, userId, and sessionId. 
#
# The primary key is composed of a composite partition key (userId, sessionId) and itemInSession as clustering columns.
# Although userId and sessionId were used in WHERE clause of Query2, including itemInSession in clustering columns
# help to sort the result (heard songs) by itemInSession.
query = """CREATE TABLE IF NOT EXISTS user_history (
    userId INT,
    sessionId INT, 
    itemInSession INT,
    firstName TEXT,
    lastName TEXT,
    artist TEXT,
    song TEXT,
    PRIMARY KEY ((userId, sessionId), itemInSession)
)
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
# Read the collected event data file to insert data into table user_history for answering query 2
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Query to insert data into user_history table
        query = "INSERT INTO user_history (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

        # Choose userId (col#10), sessionId (col#8), itemInSession (col#3), artist (col#0), song (col#9), 
        # firstName (col#1) and lastName (col#4).
        # Also convert string into int or float to match with column's data type.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
# Now run Query 2 to verify the data was entered into the table.
# Note: itemInSession is not required in show the result, including this column to verify that the result is sorted
# as Query 2's requirement.
query = "SELECT artist, song, itemInSession, firstName, lastName FROM user_history WHERE userId=10 AND sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", iteminsession=0, firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', iteminsession=1, firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', iteminsession=2, firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', iteminsession=3, firstname='Sylvie', lastname='Cruz')


### Query 3 Description

I named the table created for this query `users_by_song` to reflect the data modelled in this table that is the history of who were listening which song.

In this query, I used `song` as the partition key and `userId` as clustering key. Each partition is uniquely identified `song` while `userId` was used to uniquely identify the rows within a partition to sort users listened to the song.

In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Create table for Query 3
# The data modelled in this table is the history of who were listening which song.
#
# This table contains data columns required for query 3, including:
#   song, sessionId, firstName, and lastName. 
#
# The primary key is composed of song as partition key with userId as clustering column.
# In the sake of modeling users listening per song, song title and userId would uniquely identify each row.
query = """CREATE TABLE IF NOT EXISTS users_by_song (
    song TEXT,
    userId INT,
    firstName TEXT,
    lastName TEXT,
    PRIMARY KEY (song, userId)
)
"""
try:
    session.execute(query)
except Exception as e:
    print(e)            

In [15]:
# Read the collected event data file to insert data into table users_by_song for answering query 3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Query to insert data into users_by_song table
        query = "INSERT INTO users_by_song (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"

        # Choose song (col#9), userId (col#10), firstName (col#1) and lastName (col#4).
        # Also convert string into int or float to match with column's data type.
        values = (line[9], int(line[10]), line[1], line[4])
        #print(values)
        session.execute(query, values)

In [16]:
# Now run Query 3 to verify the data was entered into the table.
query = "SELECT firstName, lastName FROM users_by_song WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions
try:
    session.execute("DROP TABLE song_history")
except Exception as e:
    print(e)

In [18]:
try:
    session.execute("DROP TABLE user_history")
except Exception as e:
    print(e)


In [19]:
try:
    session.execute("DROP TABLE users_by_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()